In [1]:
import networkx as nx

In [210]:
import random
import numpy as np
import pandas as pd

In [77]:
import altair as alt
import nx_altair

In [38]:
def alnumspace(string):
    return string.isalnum() or string==' '

In [120]:
stopwords = "i|me|my|myself|we|us|our|ours|ourselves|you|your|yours|yourself|yourselves|he|him|his|himself|she|her|hers|herself|it|its|itself|they|them|their|theirs|themselves|what|which|who|whom|whose|this|that|these|those|am|is|are|was|were|be|been|being|have|has|had|having|do|does|did|doing|will|would|should|can|could|ought|i'm|you're|he's|she's|it's|we're|they're|i've|you've|we've|they've|i'd|you'd|he'd|she'd|we'd|they'd|i'll|you'll|he'll|she'll|we'll|they'll|isn't|aren't|wasn't|weren't|hasn't|haven't|hadn't|doesn't|don't|didn't|won't|wouldn't|shan't|shouldn't|can't|cannot|couldn't|mustn't|let's|that's|who's|what's|here's|there's|when's|where's|why's|how's|a|an|the|and|but|if|or|because|as|until|while|of|at|by|for|with|about|against|between|into|through|during|before|after|above|below|to|from|up|upon|down|in|out|on|off|over|under|again|further|then|once|here|there|when|where|why|how|all|any|both|each|few|more|most|other|some|such|no|nor|not|only|own|same|so|than|too|very|say|says|said|shall"
stopwords = set(stopwords.upper().split('|'))
wordlist = "Declarative visualization grammars can accelerate development, facilitate retargeting across platforms, and allow language-level optimizations. However, existing declarative visualization languages are primarily concerned with visual encoding, and rely on imperative event handlers for interactive behaviors. In response, we introduce a model of declarative interaction design for data visualizations. Adopting methods from reactive programming, we model low-level events as composable data streams from which we form higher-level semantic signals. Signals feed predicates and scale inversions, which allow us to generalize interactive selections at the level of item geometry (pixels) into interactive queries over the data domain. Production rules then use these queries to manipulate the visualization’s appearance. To facilitate reuse and sharing, these constructs can be encapsulated as named interactors: standalone, purely declarative specifications of interaction techniques. We assess our model’s feasibility and expressivity by instantiating it with extensions to the Vega visualization grammar. Through a diverse range of examples, we demonstrate coverage over an established taxonomy of visualization interaction techniques.We present Reactive Vega, a system architecture that provides the first robust and comprehensive treatment of declarative visual and interaction design for data visualization. Starting from a single declarative specification, Reactive Vega constructs a dataflow graph in which input data, scene graph elements, and interaction events are all treated as first-class streaming data sources. To support expressive interactive visualizations that may involve time-varying scalar, relational, or hierarchical data, Reactive Vega’s dataflow graph can dynamically re-write itself at runtime by extending or pruning branches in a data-driven fashion. We discuss both compile- and run-time optimizations applied within Reactive Vega, and share the results of benchmark studies that indicate superior interactive performance to both D3 and the original, non-reactive Vega system.We present Vega-Lite, a high-level grammar that enables rapid specification of interactive data visualizations. Vega-Lite combines a traditional grammar of graphics, providing visual encoding rules and a composition algebra for layered and multi-view displays, with a novel grammar of interaction. Users specify interactive semantics by composing selections. In Vega-Lite, a selection is an abstraction that defines input event processing, points of interest, and a predicate function for inclusion testing. Selections parameterize visual encodings by serving as input data, defining scale extents, or by driving conditional logic. The Vega-Lite compiler automatically synthesizes requisite data flow and event handling logic, which users can override for further customization. In contrast to existing reactive specifications, Vega-Lite selections decompose an interaction design into concise, enumerable semantic units. We evaluate Vega-Lite through a range of examples, demonstrating succinct specification of both customized interaction methods and common techniques such as panning, zooming, and linked selection."
wordlist = [c for c in filter(alnumspace, wordlist)]
wordlist = ''.join(wordlist).split(' ')
wordlist = [word.upper() for word in wordlist if word.upper() not in stopwords]

In [121]:
from collections import Counter
word_counts = Counter(wordlist)

In [296]:
wc = pd.DataFrame().from_dict(word_counts, orient='index').reset_index()
wc.columns = ['word', 'count']
wcfiltered = wc[wc['count']>1]

In [323]:
def shuffled_range(n): return random.sample(range(n), k=n)
n = len(wcfiltered)
x = shuffled_range(n)
y = shuffled_range(n)
data = wcfiltered.assign(x=x,y=y)

In [324]:
base = alt.Chart(data).encode(
    x=alt.X('x:Q', axis=None),
    y=alt.Y('y:Q', axis=None)
).configure_view(strokeWidth=0)


text = base.mark_text(baseline='middle').encode(
    text='word:N',
    color=alt.Color('count:Q',
                    scale=alt.Scale(scheme='goldred'),
                   ),
    size=alt.Size('count:Q', legend=None)
)

text

alt.Chart(...)

## set up graph

In [31]:
nodes = [str(n) for n in range(20)]

edges = []
for n in range(15):
    start = np.random.choice(nodes)
    end = np.random.choice(list(set(nodes) - {start}))
    edges += [(str(start), str(end))]
    
G = nx.Graph()
G.add_nodes_from(node_list)
G.add_edges_from(edges)

pos = nx.spring_layout(G)

add words to graph

In [328]:
for edge in G.edges:
    G.edges[edge]['words'] = np.random.choice(wordlist, size=np.random.randint(8), replace=False)

In [514]:
def get_graph_dfs(G, pos):
    node_data = pd.DataFrame.from_dict(pos).transpose()
    node_data.columns = 'x', 'y'
    node_data['node_id'] = layout_data.index
    
    def get_node_position(end, i):
        def f(d): return d[end].apply(lambda r: pos[r][i])
        return f

    edge_data = nx.to_pandas_edgelist(G).assign(
        x=get_node_position('source', 0),
        y=get_node_position('source', 1),
        x2=get_node_position('target', 0),
        y2=get_node_position('target', 1)
    )
    
    return node_data, edge_data

In [515]:
node_data, edge_data = get_graph_dfs(G, pos)

In [545]:
brush = alt.selection(type='interval')

nodes = alt.Chart(node_data).mark_point(
    size=100, 
).encode(
    x='x',
    y='y',
    color=alt.condition(brush, 'x', alt.value('grey'))
)

labels = nodes.mark_text(
    align='left',
    baseline='middle',
    dx=7
).encode(
    text='node_id'
)

edges = alt.Chart(edge_data).mark_line().encode(
    alt.X('x', axis=None),
    alt.Y('y', axis=None),
    alt.X2('x2'),
    alt.Y2('y2'),
)



graph_plot = (edges + nodes + labels).add_selection(brush)
graph_plot

alt.LayerChart(...)

In [ ]:
alt.Chart().mark_point()